In [ ]:
import os, random
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data, datasets
from torchtext.legacy.data import BucketIterator
from torchtext.legacy import data
from torchtext.legacy.data import TabularDataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Transformer/data"

Mounted at /content/drive


## 1. Load Dataset and Split

In [ ]:
df = pd.read_csv(os.path.join(path,'IMDb_Reviews.csv'))
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [ ]:
print(len(df))

50000


In [ ]:
train_df = df[:int(len(df)*0.8)]
test_df = df[int(len(df)*0.8):]

train_df.to_csv(os.path.join(path, "IMDb_train_data.csv"), index=False)
test_df.to_csv(os.path.join(path, "IMDb_test_data.csv"), index=False)

## 2. Define TorchText Field 

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.
- include_lengths : 토크나이즈된 문장의 길이를 같이 return 한다(True 또는 False)

In [ ]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=lambda x: x.split(),
                  lower=True,
                  batch_first=True,
                  include_lengths=True)

LABEL = data.LabelField(dtype = torch.float)

# LABEL = data.Field(sequential=False,
#                    use_vocab=False,
#                    batch_first=False,
#                    is_target=True)

## 3. Create Dataset

In [ ]:
train_data, test_data = TabularDataset.splits(
    path=path,
    train='IMDb_train_data.csv', test='IMDb_test_data.csv', format='csv',
    fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

train_data, valid_data = train_data.split(random_state = random.seed(1234))

In [ ]:
print(vars(train_data[0]))

{'text': ['this', 'is', 'the', 'finest', 'film', 'ever', 'made', 'to', 'deal', 'with', 'the', 'subject', 'of', 'aids.', "it's", 'a', 'documentary', 'about', 'two', 'men', 'living', 'with', 'and', 'dying', 'of', 'this', 'illness.', 'the', 'film', 'is', 'beautiful,', 'heartbreaking,', 'funny,', 'and', 'incredibly', 'moving.', 'above', 'all,', 'it', 'is', 'an', 'amazing', 'true', 'love', 'story.', 'be', 'sure', 'to', 'have', 'a', 'few', 'hankies', 'ready', 'before', 'you', 'watch', 'this', 'movie---you', 'will', 'need', 'them.', 'extraordinary.'], 'label': '1'}


In [ ]:
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x7f2a5bf1cf50>), ('label', <torchtext.legacy.data.field.LabelField object at 0x7f2a5bf1cfd0>)])


## 4. Create Vocabulary

- torchtext에서 제공하는 pretrained language model이 있다. 그 중 glove를 사용한다. 
- unk_init은 unk 토큰을 어떻게 initialize할지를 나타낸다. 

In [ ]:
TEXT.build_vocab(train_data,
                min_freq=10, 
                max_size=1000)
                 #  vectors = "glove.6B.100d",)

LABEL.build_vocab(train_data)

In [ ]:
print(len(TEXT.vocab))

1002


In [ ]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f2a2e46afd0>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'it': 11, 'that': 12, '/><br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'are': 22, 'not': 23, 'you': 24, 'film': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'all': 35, 'who': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'about': 42, 'her': 43, "it's": 44, 'if': 45, 'some': 46, 'out': 47, 'what': 48, 'when': 49, 'very': 50, 'there': 51, 'more': 52, 'would': 53, 'even': 54, 'my': 55, 'good': 56, 'she': 57, 'only': 58, 'no': 59, 'their': 60, 'really': 61, 'had': 62, 'up': 63, 'can': 64, 'which': 65, 'see': 66, 'were': 67, 'than': 68, '-': 69, 'we': 70, 'been': 71, 'get': 72, 'into': 73, 'will': 74, 'much': 75, 'because': 76, 'story': 77, 'most': 78, 'how': 7

## 5. DataLoader for TorchText

In [ ]:
BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    device = device)

In [ ]:
batch = next(iter(train_iterator))
print(type(batch))
print(batch)
print(batch.text)

<class 'torchtext.legacy.data.batch.Batch'>

[torchtext.legacy.data.batch.Batch of size 16]
	[.text]:('[torch.cuda.LongTensor of size 16x880 (GPU 0)]', '[torch.cuda.LongTensor of size 16 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 16 (GPU 0)]
(tensor([[  8,   2, 582,  ...,  43,   0,   0],
        [ 10,  25,   0,  ...,   1,   1,   1],
        [ 10,  14, 723,  ...,   1,   1,   1],
        ...,
        [ 78,  53,   0,  ...,   1,   1,   1],
        [ 49,   3,   0,  ...,   1,   1,   1],
        [172,   5, 695,  ...,   1,   1,   1]], device='cuda:0'), tensor([880, 853, 843, 837, 815, 807, 799, 784, 779, 777, 766, 761, 755, 748,
        746, 746], device='cuda:0'))


## 6. Model Architecture

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))        
        #embedded = [sent len, batch size, emb dim]
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'), batch_first=True)        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,410,857 trainable parameters


In [ ]:
# pretrained_embeddings = TEXT.vocab.vectors

# print(pretrained_embeddings.shape)
# model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5132, -0.0830, -0.4850,  ..., -0.3438,  0.5329,  1.7657],
        ...,
        [-0.0773,  0.3457, -0.3664,  ..., -0.6953, -0.5017, -1.1514],
        [-0.6136, -1.0967, -0.1012,  ...,  0.0047,  1.4294, -1.8340],
        [-0.1804, -0.1241, -1.0578,  ...,  0.3118,  0.0476, -1.8583]])


## 7. Train & Evaluate Model

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), os.path.join(path, 'rnn-imdb-sentiment.pt'))
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 17s
	Train Loss: 0.683 | Train Acc: 55.24%
	 Val. Loss: 0.691 |  Val. Acc: 53.02%
Epoch: 02 | Epoch Time: 2m 17s
	Train Loss: 0.637 | Train Acc: 62.06%
	 Val. Loss: 0.599 |  Val. Acc: 63.83%
Epoch: 03 | Epoch Time: 2m 17s
	Train Loss: 0.478 | Train Acc: 77.03%
	 Val. Loss: 0.430 |  Val. Acc: 80.08%
Epoch: 04 | Epoch Time: 2m 17s
	Train Loss: 0.400 | Train Acc: 81.77%
	 Val. Loss: 0.375 |  Val. Acc: 83.58%
Epoch: 05 | Epoch Time: 2m 17s
	Train Loss: 0.367 | Train Acc: 83.70%
	 Val. Loss: 0.366 |  Val. Acc: 84.46%


In [ ]:
model.load_state_dict(torch.load(os.path.join(path, 'rnn-imdb-sentiment.pt')))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')